# Building a neural net with (just) numpy

In this notebook, I aim to build a neural net with just numpy. This means in practice that I will implement the following mathemetical concepts:
* matrix multiplication (just using numpy)
* differentiation

These in turn will provide the foundations for higher level concepts such as:
* feedforward pass
* loss function calculation
* backpropagation

The following type of artificial neurons will be coded
* linear model
* Relu
* Mean Squared Error (MSE)
* Softmax

In [ ]:
# Let's start by importing our core libraries
import numpy as np
from typing import List, Dict

# Dataset selection - MNIST

We will run our neural network against the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset. MNIST is a large dataset of handwritten digits. It's commonly used as a starting point for people learning about neural networks. 

These days MNIST is considered a toy dataset because a simple, non-deep network can get you over 99% accuracy. Nevertheless, it is the perfect starting point for our experiments.

We will also try to start from scratch here and import the gzipped dataset

## Downloading MNIST (Training set only)

In this section we're only going to import the MNIST training set. 
We will import the test set a bit later.

In [ ]:
from urllib.request import urlretrieve
import gzip

In [ ]:
def download_dataset(url: str, target_file_name: str):
    urlretrieve(url, target_file_name)

In [ ]:
mnist_train_images_url = "https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz"
mnist_train_images_file_name = "data/mnist_train_data.gz"

In [ ]:
download_dataset(mnist_train_images_url, mnist_train_images_file_name)

In [ ]:
# 784 pixels in total per image
image_w = 28
image_h = 28
full_image_dimensions = image_w * image_h

In [ ]:
def load_images(file_path: str, w, h):
    """
    Loads the images from the dataset into a numpy array
    """
    with gzip.open(file_path, "rb") as f:
        raw = np.frombuffer(f.read(), np.uint8, offset=16)

    return raw.reshape(-1, w, h)

In [ ]:
train_x = load_images(mnist_train_images_file_name, image_w, image_h)

In [ ]:
train_x.shape

In [ ]:
a = train_x[0]
a.shape

In [ ]:
def load_labels(file_path: str):
    """
    Load the labels from the file path
    """
    with gzip.open(file_path, "rb") as f:
        raw = np.frombuffer(f.read(), np.uint8, offset=8)

    return raw

In [ ]:
mnist_train_labels_url = "https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz"
mnist_train_labels_file_name = "data/mnist_train_labels.gz"

In [ ]:
download_dataset(mnist_train_labels_url, mnist_train_labels_file_name)

In [ ]:
train_y = load_labels(mnist_train_labels_file_name)

In [ ]:
train_y.shape

In [ ]:
# Let's see the unique values in the labels - there should be 10 (from 0 to 9)
np.unique(train_y)

## Visualising MNIST data

Let's see what our dataset looks like

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
def visualise_data(imgs, labels, rows: int, cols: int):
    """
    Plots a grid-like visualisation of the data
    """
    figure = plt.figure(figsize=(8,8))

    for i in range(1, cols * rows + 1):
        # randomly sample from the dataset
        sampled_idx = np.random.randint(0, len(imgs))
        img, label = imgs[sampled_idx], labels[sampled_idx]
        figure.add_subplot(rows, cols, i)
        plt.title(f"{label}")
        plt.axis("off")
        plt.imshow(img, cmap="gray")

    plt.show()


In [ ]:
visualise_data(train_x, train_y, 4, 4)

## Splitting training and validation data

We're going to take 20% of the training data and keep it as the validation set.

In [ ]:
total_train_and_val_size = train_x.shape[0]
train_size = int(total_train_and_val_size * 0.8)
val_size = total_train_and_val_size - train_size

print(f"{total_train_and_val_size=} | {train_size=} | {val_size=}")

In [ ]:
# Let's pick the indices for the elements that will make up our validation set at random
val_size_indices = np.random.choice(range(total_train_and_val_size), size=val_size, replace=False)

In [ ]:
len(val_size_indices)

In [ ]:
val_x = train_x[val_size_indices]
val_y = train_y[val_size_indices]

In [ ]:
# Let's show some of the data from the validation set
visualise_data(val_x, val_y, 3, 3)

In [ ]:
# Now remove elements from the training set out of the validation set
train_x = np.delete(train_x, val_size_indices, axis=0)
train_y = np.delete(train_y, val_size_indices, axis=0)

In [ ]:
train_x.shape, train_y.shape

# Building a neural network

Our goal is to build a neural network that classifies an input image according to 10 classes representing the digits from 0 to 9. 
We'll start with a single linear layer that will predict attempt to predict the correct class.

We'll also implement ReLU and a loss function.

## A simple linear layer

In [ ]:
class Linear():
    def __init__(self, w_dim: int, b_dim: int):
        super().__init__()
        # Initialise our weight and bias matrices
        self.__w = np.random.rand(w_dim[0], w_dim[1])
        self.__b = np.random.rand(b_dim)

    def forward(self, x: np.array):
        # Keep the x - we will need it for differentiation later
        self.__x = x

        # We are reshaping our component matrices so that they are compatible
        # by default x is of shape (batch_size, 784) and w is of shape (N, 784)
        # we turn w into a matrix of dimensions (1, N, 784) and x adopts dimensions (batch_size, 784, 1)
        # this makes our matrices compatible because numpy will use broadcasting to perform the matrix multiplication
        # This will result in a matrix of dimensions (batch_size, N, 1)
        self.__w_x = self.__w[None,] @ self.__x[:, :, None]

        # Remember that b is a vector of just N elements
        # Remove the last dimension if it is one
        self.__z = (self.__w_x + self.__b[:,None]).squeeze(axis=-1)

        return self.__z


In order to be able to output a vector z with 10 columns (basically predictions for digits from 0 to 9), we need to flatten our input parameter `x` from dimensions `28 x 28` to a single vector to dimensions `784`.

Our weight matrix `W` thus will have dimensions `10 x 784`. We will perform some clever dimensional reshaping so that we can achieve a `10 x 1` output matrix for the product between `W` and `x`.

In [ ]:
w = np.random.rand(10, 784)

In [ ]:
w.shape

In [ ]:
w[None,].shape

In [ ]:
(w[None,] @ train_x.reshape(-1, 784)[:, :, None]).shape

In [ ]:
lin = Linear((10, 784), 10)

In [ ]:
lin_out = lin.forward(train_x.reshape(-1, 784))

In [ ]:
lin_out.shape

## A simple ReLU layer

Next we're going to implement a simple relu layer to clip negative activations to 0 and improve convergence to a solution.


In [ ]:
class ReLU():
    def __init__(self):
        super().__init__()

    def forward(self, x):
        self.__x = x
        self.__relu_x = self.__x.copy()
        self.__relu_x[self.__relu_x < 0 ] = 0

        return self.__relu_x

In [ ]:
rel = ReLU()

In [ ]:
rel_out = rel.forward(lin_out)

## A MSE loss layer

The final layer is that of our loss function. Here we are starting out with the Mean Squared Error, which is relatively easy to compute.

In [ ]:
class MSE():
    def __init__(self):
        super().__init__()

    def forward(self, x, y):
        self.__x = x
        self.__y = y

        self.__loss = np.mean((self.__x - self.__y) ** 2)
        return self.__loss

In [ ]:
# We need to expand our labels vector so that it has 10 columns - this is called one-hot encoding
train_y_expanded = np.zeros((train_y.shape[0], 10))
train_y_expanded[np.arange(train_y.shape[0]),train_y] = 1
train_y_expanded

In [ ]:
# We need to expand our labels vector so that it has 10 columns - this is called one-hot encoding
val_y_expanded = np.zeros((val_y.shape[0], 10))
val_y_expanded[np.arange(val_y.shape[0]), val_y] = 1
val_y_expanded

In [ ]:
# Let's check a couple of values to make sure that our 1s are at the correct indices in train_y_expanded
train_y_expanded[0:4], train_y[0:4]

In [ ]:
rel_out.shape

In [ ]:
# Now let's try out our MSE
mse = MSE()
loss = mse.forward(rel_out, train_y_expanded)

In [ ]:
loss

## Implementing gradient descent

Right now we are able to perform the forward pass, but we're missing the critical pieces required to provide feedback to our network so that it may learn how to make better predictions. To do so we need to find a way to update the weights of our layers.

To find a way to update the weights of our layer, we need to compute the partial derivatives with respect to the input and output terms for each layer, and then update the weights in the opposite direction to the gradient. This is in essence what the backpropagation algorithm does.

The intuitive reasoning behind this, is because we can think of a neural network as a series of compositions of functions... From a notation standpoint we could write:

$$
\text{Network} = MSE(ReLU(Linear(x, w, b)))
$$

The above is a mathematical definition of the _forward_ pass of the network.


Now, assuming the functions in the expression above are differentiable (they are), we can find a way to determine how changing specific parameters affects the output. This is after all what differentiation allows you to discover. 

Remember also that the formula for calculating the derivative of a function composition is the following:

$$
\text{The chain rule states: If } y = f(u) \text{ and } u = g(x), \text{ then the derivative of } y \text{ with respect to } x \text{ is:}
$$
$$
\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx} 
$$

Put more succinctly, we could also write:
$$
\frac{d}{dx}h(x) = \frac{d}{dx}f(g(x)) = f'(g(x)) \cdot g'(x)
$$


There is one little we forgot to mention... We are dealing with a multi-dimensional problem that involves multiple variables. So we need to understand how changing each variable affects the overall result.

We do this by computing not a single derivative, but instead by calculating the partial derivatives with respect to a given term (e.g. with respect to $w$ or $b$, etc.).

So essentially, we apply the chain rule recursively (from the last layer of the network up until the first layer) by computing the partial derivative for the appropriate terms. This is called the _backward_ pass, since it runs in the opposite direction to the forward pass.

Well... that's exactly what backpropagation is! 


Let's thus reimplement our classes with a backward pass.

### MSE Loss layer with gradient calculation

In [ ]:
class MSEWithGrad():
    def __init__(self):
        super().__init__()

    def forward(self, x, y):
        self.__x = x
        self.__y = y

        self.__diff = self.__x - self.__y
        self.__loss = np.mean(self.__diff ** 2)
        return self.__loss

    def preds(self):
        return self.__diff ** 2

    def backward(self):
        self.__grad = 2 * self.__diff / self.__diff.shape[0]

    def gradient(self):
        return self.__grad

    def gradient_update(self, learning_rate):
        """
        MSE doesn't have learnable parameters so there is nothing to update in gradient descent
        """
        pass

    def gradient_zero(self):
        self.__grad = 0.



In [ ]:
mse_w_grad = MSEWithGrad()

In [ ]:
loss = mse_w_grad.forward(rel_out, train_y_expanded)
mse_w_grad.backward()
mse_gradient = mse_w_grad.gradient()

print(f"{loss=} | {mse_gradient=}")

In [ ]:
mse_gradient.shape

In [ ]:
class ReluWithGrad():
    def __init__(self):
        super().__init__()

    def forward(self, x):
        self.__x = x
        self.__relu_x = self.__x.copy()
        self.__relu_x[self.__relu_x < 0 ] = 0

        return self.__relu_x

    def backward(self, next_layer_grad):
        """
        Computes gradient of this layer using backpropagation
        """
        x_copy = self.__x.copy()
        self.__d_relu_dx = np.where(x_copy > 0, 1, 0)
        self.__grad = self.__d_relu_dx * next_layer_grad

    def gradient(self):
        return self.__grad

    def gradient_update(self, learning_rate):
        """
        ReLU doesn't have learnable parameters so there is nothing to update in gradient descent
        """
        pass

    def gradient_zero(self):
        self.__d_relu_dx = 0.
        self.__grad = 0.


In [ ]:
relu_w_grad = ReluWithGrad()
relu_w_grad_out = relu_w_grad.forward(lin_out)
relu_w_grad.backward(mse_gradient)
relu_w_grad_gradient = relu_w_grad.gradient()

print(f"{relu_w_grad_out.shape=} | {mse_gradient.shape=} | {relu_w_grad_gradient=}")

In [ ]:
class LinearWithGrad():
    def __init__(self, w_dim: int, b_dim: int):
        super().__init__()
        # Initialise our weight and bias matrices
        self.__w = np.random.rand(w_dim[0], w_dim[1])
        self.__b = np.random.rand(b_dim)

        print(f"{self.__w.shape=} | {self.__b.shape=}")

    def forward(self, x: np.array):
        # Keep the x - we will need it for differentiation later
        self.__x = x
        # print(f"{self.__x.shape=}")

        # We are reshaping our component matrices so that they are compatible
        # by default x is of shape (batch_size, 784) and w is of shape (10, 784)
        # we turn w into a matrix of dimensions (1, 10, 784) and x adopts dimensions (batch_size, 784, 1)
        # this makes our matrix compatible because numpy will use broadcasting to perform the matrix multiplication
        # This will result in a matrix of dimensions (batch_size, 10, 1)
        self.__w_x = self.__x @ self.__w

        # Remember that b is a vector of just 10 elements
        # Remove the last dimension if it is one
        # self.__z = (self.__w_x + self.__b[None,:]).squeeze(axis=-1)
        self.__z = (self.__w_x + self.__b[None,:])

        # print(f"{self.__z.shape=}")

        return self.__z

    def backward(self, next_layer_grad):
        self.__d_lin_dx = next_layer_grad @ self.__w.T
        self.__d_lin_dw = self.__x.T @ next_layer_grad
        self.__d_lin_db = next_layer_grad.sum(axis=0)

    def gradient(self):
        return self.gradient_inputs()

    def gradient_inputs(self):
        return self.__d_lin_dx

    def gradient_weights(self):
        return self.__d_lin_dw

    def gradient_bias(self):
        return self.__d_lin_db

    def gradient_update(self, learning_rate):
        self.__w -= self.__d_lin_dw * learning_rate
        self.__b -= self.__d_lin_db * learning_rate

    def gradient_zero(self):
        self.__d_lin_dw = 0
        self.__d_lin_db = 0
        self.__d_lin_dx = 0


In [ ]:
lin_w_grad = LinearWithGrad((784, 10), 10)
lin_w_grad_out = lin_w_grad.forward(train_x.reshape(-1, 784))
lin_w_grad.backward(relu_w_grad_gradient)

lin_w_grad_gradient_weights = lin_w_grad.gradient_weights()
lin_w_grad_gradient_bias = lin_w_grad.gradient_bias()

print(f"{lin_w_grad_gradient_weights.shape=} | {lin_w_grad_gradient_bias.shape=}")

# Putting it all together

Let's train our neural network from scratch using everything we built up until now. But first we will need to carry out a bit of pre-processing on our MNIST dataset.

In [ ]:
def preprocess_mnist(x: np.array) -> np.array:
    """
    In this step we simply normalize the values so that
    they range from [0-1] instead of [0-255].
    This will improve conversion
    """
    return x/255.0

In [ ]:
def calculate_accuracy(preds_y, y) -> float:
    """
    Returns percentage of correct predictions across the whole dataset
    """
    total_examples = y.shape[0]
    correct_predictions = 0

    for i in range(total_examples):
        highest_pred = np.argmax(preds_y[i])
        ground_truth = np.argmax(y[i])

        if highest_pred == ground_truth:
            correct_predictions += 1

    return float(correct_predictions) / float(total_examples)

In [ ]:
losses = []
epochs = 100
learning_rate = 0.01

In [ ]:
model = [LinearWithGrad((784, 10), 10), ReluWithGrad(), MSEWithGrad()]

In [ ]:
input = preprocess_mnist(train_x.reshape(-1, 784))
ground_truth = train_y_expanded
out = None
for i in range(epochs):
    inp = input
    for m_i, m in enumerate(model):
        if m_i == len(model) -1 :
            out = m.forward(inp, ground_truth)
            losses.append(out)
        else:
            out = m.forward(inp)
        inp = out

    grad = None
    for m_i, m in enumerate(reversed(model)):
        if m_i == 0:
            m.backward()
        else:
            m.backward(grad)

        grad = m.gradient()

    for m in model:
        m.gradient_update(learning_rate)
        m.gradient_zero()

In [ ]:
losses

In [ ]:
# Let's visualise our loss...
import matplotlib.pyplot as plt

In [ ]:
plt.plot(losses)
plt.ylabel("loss score")
plt.xlabel("epoch")
plt.show()

In [ ]:
# Let's see what the output looks like
preds = model[-1].preds()
preds.shape

In [ ]:
n = 500
preds[n], np.argmax(preds[n])

In [ ]:
idx = np.argmax(preds[n])
true_pred = np.argmax(train_y_expanded[n])

idx, true_pred

# Building a Softmax and Cross Entropy Loss layers

Since we are dealing with a classification problem, using Softmax the towards the end of our network is a much more appropriate choice since we can better interpret the predictions of the network with respect to the possible values of _Y_ (i.e. digits from 0 to 9).

$$ \text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}} $$

Additionally, we will be using the cross entropy loss function. Let's explain the formula for cross-entropy loss below:

$$
L(y, p) = - {\sum_{j=1}^{K} y_j  log(p(y_j))}
$$

Here $y_j$ represents the ground truth probability a given class, while $p(y_j)$ represents the predicted (by our network) probability for each label. In our case, we are dealing with a typical classification problem, where there can only be one correct class $y_i$ per example. 

Therefore it follows that $y_i = 1$.
Additionally, it also follows that $y_k = 0, k \neq i$. **This means that all ground truth probabilities for incorrect classes are 0**. So our loss function can effectively be simplified, where for each example 

$$
L(y, p) = - y_i  log(p(y_i))
$$

Since $y_i = 1$ the formula simply becomes:

$$
L(y, p) = - log(p(y_i))
$$

Interestingly, this is a special case of cross entropy called the **Negative Loss Likelihood**. Although both terms are often used interchangibly in ML circles, negative loss likelihood only makes sense if the probabilities for each correct label is 1, while those of the incorrect labels are all 0.


In [ ]:
class Softmax:
    def __init__(self):
        super().__init__()

    def forward(self, z):
        self.__numerator = np.exp(z)
        self.__denominator = np.sum(self.__numerator, axis=1, keepdims=True)
        self.__softmax = self.__numerator / self.__denominator

        return self.__softmax




In [ ]:
class CrossEntropyLoss:
    def __init__(self):
        super().__init__()

    def forward(self, z, y):
        self.__y_i_index = np.argmax(y, axis=1)
        self.__z_i = z[np.arange(z.shape[0]), self.__y_i_index]
        self.__loss_per_row = - (np.log(self.__z_i))

        # The final loss is the average loss across the whole batch
        self.__loss = self.__loss_per_row.mean()
        return self.__loss


In [ ]:
rel_out

In [ ]:
sm = Softmax()
sm_out = sm.forward(rel_out)
sm_out

## Making softmax more numerically stable

What a bummer! Looks like we overflow across most computations of our softmax layer. This is due to raising to the exponential of large numbers (over $10^4$).

There's trick to make the computations more numerically stable though... What we can do is shift our numbers to a lower range, which should make the calculations more stable numerically. To do this, we identify the largest number $m_i$ on each row (i.e. for each example) from our input $z$, and instead take the exponential of the difference between the value original value of z and our max number. 

The formula becomes thus:

$$ 
\text{softmax}(z_i) = \frac{e^{z_i - \text{max}(z)}}{\sum_{j=1}^{K} e^{z_j-\text{max}(z)}} 
$$

What's good about this new formula is that the relative order of all inputs is also preserved.

In [ ]:
class SoftmaxStable:
    def __init__(self):
        super().__init__()

    def forward(self, z):
        self.__z = z
        self.__max_z = np.max(z, axis=1, keepdims=True)
        self.__z_minus_max = z - self.__max_z

        self.__numerator = np.exp(self.__z_minus_max)
        self.__denominator = np.sum(self.__numerator, axis=1, keepdims=True)
        self.__softmax = self.__numerator / self.__denominator

        return self.__softmax

In [ ]:
sms = SoftmaxStable()
sms_out = sms.forward(rel_out)
sms_out

In [ ]:
sms_out.shape

In [ ]:
cel = CrossEntropyLoss()
cel_out = cel.forward(sms_out, train_y_expanded)
cel_out

## Making Cross Entropy Loss more numerically stable

I've just hit another roadblock 🫠. I get some infinity errors because some of the values from the softmax layer (the ones for the true probability class) are $0$ or very very close to it. **For recall $log(0)$ is undefined**, which is probably why I'm getting this issue.

What we can do to address this issue is to add a constant term _epsilon_, which will ensure that the outputs of my softmax later are never truly zero.

In [ ]:
class CrossEntropyLossStable:
    def __init__(self):
        super().__init__()
        self.__epsilon = 1e-9

    def forward(self, z, y):
        self.__y_i_index = np.argmax(y, axis=1)
        self.__z_i = z[np.arange(z.shape[0]), self.__y_i_index]
        self.__z_epislon = self.__z_i + self.__epsilon
        self.__loss_per_row = - (np.log(self.__z_epislon))

        # The final loss is the average loss across the whole batch
        self.__loss = self.__loss_per_row.mean()
        return self.__loss


In [ ]:
cels = CrossEntropyLossStable()
cels_out = cels.forward(sms_out, train_y_expanded)
cels_out

# Putting it all together (2)

Now that we can compute the Softmax and Cross Entropy, we need to implement backpropagation logic for these two layers.



## Gradient of cross entropy layer

The gradient of the cross entropy loss function is simply the difference between the predicted loss batch and the ground truth for each item in the batch. From a notation standpoint, we could write the following:
$$
\frac{\partial \mathcal{L}}{\partial z_i} = p_i - y_i
$$

Let's explain this formula in more detail:
* $\mathcal{L}$ represents the cross entropy loss function
* $z_i$ represents the logits; i.e. inputs into the Softmax function
* $p_i$ is the predicted probabilities for class _i_, which is produced from the Softmax function
* $y_i$ is the ground truth; i.e. the actual probability for class $i$ (either 1 or 0 depending on whether $i$ is the true class)
* $\frac{\partial \mathcal{L}}{\partial z_i}$ is thus the partial derivative of the loss function with respect to the logit $z_i$

In [ ]:
class CrossEntropyLossStableWithGradient:
    def __init__(self):
        super().__init__()
        self.__epsilon = 1e-9

    def forward(self, z, y):
        self.__y = y
        self.__z = z
        self.__y_i_index = np.argmax(y, axis=1)
        self.__z_i = z[np.arange(z.shape[0]), self.__y_i_index]
        self.__z_epislon = self.__z_i + self.__epsilon
        self.__loss_per_row = - (np.log(self.__z_epislon))

        # The final loss is the average loss across the whole batch
        self.__loss = self.__loss_per_row.mean()
        return self.__loss

    def backward(self):
        self.__d_loss_z = self.__z - self.__y

    def gradient(self):
        return self.__d_loss_z

    def gradient_update(self, learning_rate):
        # Nothing to update since Cross Entropy doesn't have hidden layers
        pass

    def preds(self):
        return self.__z

    def gradient_zero(self):
        self.__d_loss_z = 0


## Gradient of Softmax layer

Calculating the gradient of the Softmax layer is a lot more involved. We could use a little trick to simplify it, assuming that Softmax is combined with cross entropy at the end. But for the purpose of this exercise, we won't make this assumption.

The first thing we need to do is compute the **Jacobian** for the Softmax layer. The Jacobian represents the matrix of first order partial derivatives. 

Remember that for any item in the batch, the formula for Softmax for a given class $i$ (assuming $K$ classes) is the following 

$$
\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}
$$

We can now think about how to calculate the Jacobian for the expression above... Strap on to your seats, because this is going to get exciting.

---

### Calculating the Jacobian of Softmax

Since we are dealing with a batch, we can expand this formula to a matrix form. Let's assume our batch contains 3 examples, and we have 3 classes per batch.

In this example, the output of the forward pass for the Softmax layer thus becomes the following matrix:

$$
\text{S} = \begin{pmatrix}
\frac{e^{z_{11}}}{\sum_{j=1}^{3} e^{z_{1j}}} & \frac{e^{z_{12}}}{\sum_{j=1}^{3} e^{z_{1j}}} & \frac{e^{z_{13}}}{\sum_{j=1}^{3} e^{z_{1j}}} \\
\frac{e^{z_{21}}}{\sum_{j=1}^{3} e^{z_{2j}}} & \frac{e^{z_{22}}}{\sum_{j=1}^{3} e^{z_{2j}}} & \frac{e^{z_{23}}}{\sum_{j=1}^{3} e^{z_{2j}}} \\
\frac{e^{z_{31}}}{\sum_{j=1}^{3} e^{z_{3j}}} & \frac{e^{z_{32}}}{\sum_{j=1}^{3} e^{z_{3j}}} & \frac{e^{z_{33}}}{\sum_{j=1}^{3} e^{z_{3j}}} \\
\end{pmatrix}
$$

We can further adopt a more compact notation for the above matrix with the expression below:
$$
\text{S} = \begin{pmatrix}
S_{11} & S_{12} & S_{13} \\
S_{21} & S_{22} & S_{23} \\
S_{31} & S_{22} & S_{33} \\
\end{pmatrix}
$$

For each item in the batch from the formula above, **we can see that we need to compute 3 partial derivatives for any input $z_i$ because we have 3 classes ($K = 3$)** .

So this means in practical terms, for input $z_{1i}$ (i.e. first item in the batch), its Jacobian is the following:

$$
\frac{\partial S_{1}}{\partial z_{1}} = \begin{pmatrix}
\frac{\partial S_{11}}{\partial z_{11}} & \frac{\partial S_{12}}{\partial z_{11}} & \frac{\partial S_{13}}{\partial z_{11}} \\
\frac{\partial S_{11}}{\partial z_{12}} & \frac{\partial S_{12}}{\partial z_{12}} & \frac{\partial S_{13}}{\partial z_{12}} \\
\frac{\partial S_{11}}{\partial z_{13}} & \frac{\partial S_{12}}{\partial z_{13}} & \frac{\partial S_{13}}{\partial z_{13}} \\
\end{pmatrix}
$$

With the above, it follows that **for any single item in the batch (i.e. a row), its Jacobian is a square matrix of $K \times K$ dimensions, where K is the number of classes**. So this means that in our example above, the Jacobian for the first item in the batch is a $3 \times 3$ matrix. Furthermore, it follows that the Jacobian for the full match is a matrix of $B \times K \times K$ dimensions. In our example where $B = 3$ and $K = 3$, our Jacobian is a $3 \times 3 \times 3$ matrix.

---

#### Calculating the partial derivatives with respect to one logit

To derive a more generic formula for the partial derivatives, let's look at how we may compute the 3 partial derivatives with respect to $z_{11}$. 

$$
\begin{align}
\frac{\partial S_{11}}{\partial z_{11}} &= \frac{\partial}{\partial z_{11}} \frac{e^{z_{11}}}{\sum_{j=1}^{K} e^{z_{1k}}} \\
\end{align}
$$

To compute such a derivative, we employ the *quotient rule*, expressed in the below:
$$
\frac{d}{dx} \left( \frac{u}{v} \right) = \frac{v \frac{du}{dx} - u \frac{dv}{dx}}{v^2}
$$


##### **First partial derivative**
So plugging in all we know, the partial derivative of $S_{11}$ with respect to $z_{11}$ is:


$$
\begin{align}
\frac{\partial S_{11}}{\partial z_{11}} &= \frac{\partial}{\partial z_{11}} \frac{e^{z_{11}}}{\sum_{j=1}^{K} e^{z_{1k}}} \\
&= \frac{e^{z_{11}} \cdot (e^{z_{11}} + e^{z_{12}} + e^{z_{13})} - e^{z_{11}} \cdot e^{z_{11}} }{(e^{z_{11}} + e^{z_{12}} + e^{z_{13})^2}} \\
&= \frac{e^{z_{11}} \cdot [(e^{z_{11}} + e^{z_{12}} + e^{z_{13}}) - e^{z_{11}}] }{(e^{z_{11}} + e^{z_{12}} + e^{z_{13})^2}} \\
&= \frac{e^{z_{11}} \cdot (e^{z_{11}} + e^{z_{12}} + e^{z_{13}})}{(e^{z_{11}} + e^{z_{12}} + e^{z_{13})^2}} - \frac{e^{z_{11}} \cdot e^{z_{11}}}{(e^{z_{11}} + e^{z_{12}} + e^{z_{13})^2}} \\
&= \frac{e^{z_{11}}}{e^{z_{11}} + e^{z_{12}} + e^{z_{13}}} - (\frac{e^{z_{11}}}{e^{z_{11}} + e^{z_{12}} + e^{z_{13}}})^2 \\
&= S_{11} - (S_{11})^2 \\
&= S_{11} \cdot (1 - S_{11}) \\
\end{align}
$$

Expression the above as a one-line formula, we have:
$$
\begin{align}
\frac{\partial S_{11}}{\partial z_{11}} &= S_{11} \cdot (1 - S_{11}) \\
\end{align}
$$

##### **Second partial derivative**

Now let's move on to the second partial derivative, still with respect to $z_{11}$


$$
\begin{align}
\frac{\partial S_{12}}{\partial z_{11}} &= \frac{\partial}{\partial z_{11}} \frac{e^{z_{12}}}{\sum_{j=1}^{K} e^{z_{1k}}} \\
&= \frac{0 - e^{z_{11}} \cdot e^{z_{12}} }{(e^{z_{11}} + e^{z_{12}} + e^{z_{13}})^2} \\
&= - (\frac{e^{z_{11}}}{(e^{z_{11}} + e^{z_{12}} + e^{z_{13}})} \cdot \frac{e^{z_{12}}}{(e^{z_{11}} + e^{z_{12}} + e^{z_{13}})}) \\
&= - (S_{11} \cdot S_{12})
\end{align}
$$

Expressing the above as a one-line formula, we have:
$$
\begin{align}
\frac{\partial S_{12}}{\partial z_{11}} &= - (S_{11} \cdot S_{12}) \\
\end{align}
$$


##### **Third partial derivative**

Finally, the third partial derivative, still with respect to $z_{11}$, is expressed is calculated as follows:

$$
\begin{align}
\frac{\partial S_{13}}{\partial z_{11}} &= \frac{\partial}{\partial z_{11}} \frac{e^{z_{13}}}{\sum_{j=1}^{K} e^{z_{1k}}} \\
&= \frac{0 - e^{z_{11}} \cdot e^{z_{13}} }{(e^{z_{11}} + e^{z_{12}} + e^{z_{13}})^2} \\
&= - (\frac{e^{z_{11}}}{(e^{z_{11}} + e^{z_{13}} + e^{z_{13}})} \cdot \frac{e^{z_{13}}}{(e^{z_{11}} + e^{z_{12}} + e^{z_{13}})}) \\
&= - (S_{11} \cdot S_{13})
\end{align}
$$

Expressing the above as a one-line formula, we have:
$$
\begin{align}
\frac{\partial S_{13}}{\partial z_{11}} &= - (S_{11} \cdot S_{13}) \\
\end{align}
$$

#### General rules for partial derivative

We can see from the above that we can derive the partial derivative with respect to any term using the following formulas

$$
\begin{align}
\frac{\partial S_j}{\partial z_{i}} &= S_{i} \cdot (1 - S_{i}), i = j \\
\end{align}
$$

$$
\frac{\partial S_j}{\partial z_{i}} = - (S_{i} \cdot S_{j}), i \neq j \\
$$

As result, this means we could rewrite the expression above for the Jacobian of for the first item in our example batch of 3 classes as the following:

$$
\frac{\partial S_{1}}{\partial z_{1}} = \begin{pmatrix}
S_{11} \cdot (1 - S_{11}) & - (S_{11} \cdot S_{12}) & - (S_{11} \cdot S_{13}) \\
-(S_{12} \cdot S_{11}) & S_{12} \cdot (1 - S_{12}) & -(S_{12} \cdot S_{13}) \\
-(S_{13} \cdot S_{11}) & -(S_{13} \cdot S_{12}) & S_{13} \cdot (1 - S_{13}) \\
\end{pmatrix}
$$

See how the diagonal elements match the case where $i=j$, while all off-diagonal elements match the case where $i \neq j$.

With this knowledge, we can go on to code the full Jacobian in our Softmax layer...



In [ ]:
class SoftmaxStableWithGradient:
    def __init__(self):
        super().__init__()

    def forward(self, z):
        self.__z = z
        self.__max_z = np.max(z, axis=1, keepdims=True)
        self.__z_minus_max = z - self.__max_z

        self.__numerator = np.exp(self.__z_minus_max)
        self.__denominator = np.sum(self.__numerator, axis=1, keepdims=True)
        self.__softmax = self.__numerator / self.__denominator

        return self.__softmax

    def backward(self, next_layer_grad):
        # Initialise the Jacobian - it's a 3D array where for each sample in the batch we compute a 2D array
        batch_size, num_classes = self.__softmax.shape
        self.__d_softmax_z = np.zeros((batch_size, num_classes, num_classes))
        # Now iterate through each element to create the Jacobian
        for i in range(batch_size):
            diag_i = np.diagflat(self.__softmax[i])
            jacobian_i =  - np.outer(self.__softmax[i], self.__softmax[i]) + diag_i
            self.__d_softmax_z[i] = jacobian_i

        self.__gradient = np.einsum("ij,ijk->ik", next_layer_grad,  self.__d_softmax_z)

    def preds(self):
        return self.__softmax

    def gradient(self):
        return self.__gradient

    def gradient_update(self, learning_rate):
        # Nothing to update since softmax doesn't have hidden layers
        pass

    def gradient_zero(self):
        self.__gradient = 0


# Putting it all together together

Let's run our network again with the softmax and cross entropy layers and see how well it fares

In [ ]:
losses = []
epochs = 100
learning_rate = 0.0001

In [ ]:
model = [LinearWithGrad((784, 10), 10), SoftmaxStableWithGradient(), CrossEntropyLossStableWithGradient()]

In [ ]:
input = preprocess_mnist(train_x.reshape(-1, 784))
ground_truth = train_y_expanded
out = None
for i in range(epochs):
    inp = input
    for m_i, m in enumerate(model):
        if m_i == len(model) -1 :
            out = m.forward(inp, ground_truth)
            losses.append(out)
        else:
            out = m.forward(inp)
        inp = out

    grad = None
    for m_i, m in enumerate(reversed(model)):
        if m_i == 0:
            m.backward()
        else:
            m.backward(grad)

        grad = m.gradient()

    for m in model:
        m.gradient_update(learning_rate)
        m.gradient_zero()

In [ ]:
def train_model(model, epochs, learning_rate, x, y) -> (List[float], List[float]):
    """
    Trains the model for the set number of epochs.
    Returns a tuple composed of the losses per epoch along with the accuracy per epoch
    """
    input = x
    ground_truth = y
    losses = []
    accuracy = []
    out = None
    for i in range(epochs):
        inp = input
        for m_i, m in enumerate(model):
            if m_i == len(model) -1 :
                # Compute the accuracy with the predictions from the penultimate layer just before we do a forward pass to the last layer
                accuracy.append(calculate_accuracy(out, ground_truth))

                out = m.forward(inp, ground_truth)
                losses.append(out)
            else:
                out = m.forward(inp)
            inp = out

        # Now do backpropagation...
        grad = None
        for m_i, m in enumerate(reversed(model)):
            if m_i == 0:
                m.backward()
            else:
                m.backward(grad)

            grad = m.gradient()

        # Now do another forward pass where we update the layers by applying a learning rate to the gradients
        for m in model:
            m.gradient_update(learning_rate)
            # Don't forget to reset the gradients
            m.gradient_zero()

    return (losses, accuracy)

In [ ]:
def predict(model, x, y) -> np.array:
    """
    Returns the model's predictions for a given input x and ground truth y
    """
    inp = x
    out = None
    preds = None
    ground_truth = y
    for m_i, m in enumerate(model):
        if m_i == len(model) -1 :
            # predictions are made in the penultimate layer, before the loss function layer
            preds = out

            out = m.forward(inp, ground_truth)
            losses.append(out)
        else:
            out = m.forward(inp)
        inp = out

    return preds

In [ ]:
 # ,
train_x_preprocessed = preprocess_mnist(train_x.reshape(-1, 784))

In [ ]:
lr = 0.0001
m = [LinearWithGrad((784, 10),10), SoftmaxStableWithGradient(), CrossEntropyLossStableWithGradient()]
eps = 200
ls, accs = train_model(m, eps, lr, train_x_preprocessed, train_y_expanded)

In [ ]:
val_x_preprocessed = preprocess_mnist(val_x.reshape(-1, 784))

In [ ]:
val_x_preprocessed.shape, val_y_expanded.shape

In [ ]:
val_preds = predict(m, val_x_preprocessed, val_y_expanded)

In [ ]:
val_accs = calculate_accuracy(val_preds, val_y_expanded)
val_accs

In [ ]:
accs[-1]

In [ ]:
# ls

In [ ]:
# losses

In [ ]:
plt.plot(ls)
plt.ylabel("loss score")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(accs)
plt.ylabel("accuracy (train set) (%)")
plt.xlabel("epoch")
plt.show()

In [ ]:
accs[-1]

In [ ]:
# Let's see what the output looks like
preds = model[-1].preds()
preds.shape

In [ ]:
n = 1500
preds[n], np.argmax(preds[n])

In [ ]:
idx = np.argmax(preds[n])
true_pred = np.argmax(train_y_expanded[n])

idx, true_pred

In [ ]:
acc = calculate_accuracy(preds, train_y_expanded)
acc